# Import Packages

In [42]:
# Core Packages:
import pandas as pd
from pandas import DataFrame
import pickle
import numpy as np
from numpy import unique
from numpy import arange
from matplotlib import pyplot
import re
import os
import warnings
warnings.filterwarnings('ignore')

#"""
# Google Colab:
### Print Dataframe like Spreadsheet!
from google.colab import data_table
data_table.enable_dataframe_formatter()
#"""

#"""
# Mount Drive:
from google.colab import drive
drive.mount('/content/drive')
#"""

from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import textblob
from textblob import TextBlob



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
# Import Required Python Libraries:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV

#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import log_loss

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

# Hides warnings
#import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
#warnings.filterwarnings(action='ignore',module='sklearn')

In [43]:
df = pd.read_pickle("/content/drive/MyDrive/capstone_data/ready_for_modeling.pkl")

In [44]:
df.head()

,description,neighborhood_overview,host_about,host_has_profile_pic,host_identity_verified,latitude,longitude,property_type,room_type,accommodates,...,license_bool,bathrooms,is_shared_bathroom,amenities_keep,air_conditioning,kitchen,wifi,free_park_prem,pets_allowed,pool
0,Dont pay ridiculously high cleaning fees elsew...,South slope of Brown Mountain overlooking sout...,Hello My husband and I are looking forward to ...,1,1,35.49233,-82.51495,Private room in home,Private room,3,...,0,1.0,0,"air conditioning, free parking on premises, wifi",1,0,1,1,0,0
1,Walk to town in ten minutes Monthly rental inc...,"I love my neighborhood Its friendly, easy-goin...",I am a long time resident of Asheville and am ...,1,1,35.60562,-82.55512,Entire rental unit,Entire homeapt,2,...,0,1.0,0,"kitchen, air conditioning, pets allowed, wifi,...",1,1,1,1,1,0
2,space Pet friendly private cottage located ...,"We are within easy walk of pubs, breweries, mu...",We operate a travelers hostel located in Histo...,1,1,35.57864,-82.59578,Entire guesthouse,Entire homeapt,2,...,0,1.0,0,"air conditioning, kitchen, pets allowed, wifi,...",1,1,1,1,1,0
3,space Private Rooms at Bon Paul and Sharkys...,"Easy walk to pubs, cafes, bakery, breweries, l...",We operate a travelers hostel located in Histo...,1,1,35.57864,-82.59578,Private room in home,Private room,2,...,0,2.5,1,"wifi, kitchen, free parking on premises",0,1,1,1,0,0
4,This is a top bunk in the mixed dorm room Ple...,NaN,We operate a travelers hostel located in Histo...,1,1,35.57864,-82.59578,Shared room in hostel,Shared room,1,...,0,2.5,1,"air conditioning, wifi, free parking on premises",1,0,1,1,0,0


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   description             272232 non-null  object 
 1   neighborhood_overview   167804 non-null  object 
 2   host_about              164351 non-null  object 
 3   host_has_profile_pic    274958 non-null  int64  
 4   host_identity_verified  274958 non-null  int64  
 5   latitude                274958 non-null  float64
 6   longitude               274958 non-null  float64
 7   property_type           274958 non-null  object 
 8   room_type               274958 non-null  object 
 9   accommodates            274958 non-null  int64  
 10  bedrooms                205357 non-null  float64
 11  beds                    272084 non-null  float64
 12  price                   274958 non-null  float64
 13  minimum_nights          274958 non-null  int64  
 14  maximum_nights      

# NLP Pre-Processing: Polarity using TextBlob on `description`

## Description

In [48]:
df['description'] = df['description'].fillna('')

In [49]:
df['desc_polarity'] = df['description'].apply(lambda text: TextBlob(text).polarity)
df['desc_subjectivity'] = df['description'].apply(lambda text: TextBlob(text).subjectivity)

# Pickled df having desc. polarity and subjectivity

In [76]:
df.to_pickle('desc_nlp_done.pkl')

## Neighborhood Overview

In [50]:
#df['neighborhood_overview'] = df['neighborhood_overview'].fillna('')

In [ ]:
#df['desc_polarity'] = df['description'].apply(lambda text: TextBlob(text).polarity)
#df['desc_subjectivity'] = df['description'].apply(lambda text: TextBlob(text).subjectivity)

## Host About

In [ ]:
#df['neighborhood_overview'] = df['neighborhood_overview'].fillna('')

In [ ]:
#df['desc_polarity'] = df['description'].apply(lambda text: TextBlob(text).polarity)
#df['desc_subjectivity'] = df['description'].apply(lambda text: TextBlob(text).subjectivity)

## Create df for pipeline:
- Not using for now to save computation time:
  - `property_type`

In [51]:

drop_for_pipeline = ['description', 'neighborhood_overview','host_about', 'amenities_keep', 'property_type']
df.drop(drop_for_pipeline, axis=1, inplace=True)


In [52]:
df.head()

,host_has_profile_pic,host_identity_verified,latitude,longitude,room_type,accommodates,bedrooms,beds,price,minimum_nights,...,bathrooms,is_shared_bathroom,air_conditioning,kitchen,wifi,free_park_prem,pets_allowed,pool,desc_polarity,desc_subjectivity
0,1,1,35.49233,-82.51495,Private room,3,1.0,1.0,47.0,1,...,1.0,0,1,0,1,1,0,0,0.150863,0.588912
1,1,1,35.60562,-82.55512,Entire homeapt,2,1.0,1.0,100.0,30,...,1.0,0,1,1,1,1,1,0,0.190298,0.374345
2,1,1,35.57864,-82.59578,Entire homeapt,2,1.0,1.0,100.0,1,...,1.0,0,1,1,1,1,1,0,0.130754,0.458862
3,1,1,35.57864,-82.59578,Private room,2,NaN,1.0,66.0,1,...,2.5,1,0,1,1,1,0,0,0.135000,0.427500
4,1,1,35.57864,-82.59578,Shared room,1,NaN,6.0,26.0,1,...,2.5,1,1,0,1,1,0,0,0.175000,0.515278


# Inspect Columns to OHE

In [53]:
data_table.enable_dataframe_formatter()
obj_cols = []
for col in range(df.shape[1]):
  if df[df.columns[col]].dtypes == 'object':
    obj_cols.append(df.columns[col])
print()
for col in obj_cols:
  print(col)
obj_df = df[obj_cols]
print()
obj_df.apply(lambda obj_df: obj_df.sample(5, random_state=0))


room_type
city
state



,room_type,city,state
114282,Private room,Rochester,New York
119428,Entire homeapt,New York City,New York
269296,Private room,Clark County,Nevada
196869,Entire homeapt,Austin,Texas
215411,Entire homeapt,Twin Cities,Minnesota


In [54]:
DataFrame(df['room_type'].value_counts())

,room_type
Entire homeapt,208476
Private room,63295
Shared room,2379
Hotel room,808


In [55]:
 DataFrame(df['city'].value_counts())

,city
Los Angeles,44594
New York City,38792
Hawaii,33339
Broward County,17915
Clark County,16639
Austin,14861
San Diego,12497
Nashville,8584
Chicago,8528
Santa Clara County,6853


In [56]:
 DataFrame(df['state'].value_counts())

,state
California,72079
New York,39730
Hawaii,33339
Texas,22295
Florida,17915
Nevada,16639
Tennessee,8584
Illinois,8528
Washington,6823
Louisiana,6783


# Define Modeling Class ModelWithCV()

In [87]:
class ModelWithCV():
    '''Structure to save the model and more easily see its crossvalidation'''

    def __init__(self, model, model_name, X, y, cv_now=True):
        self.model = model
        self.name = model_name
        self.X = X
        self.y = y
        # For CV results
        self.cv_results = None
        self.cv_mean = None
        self.cv_median = None
        self.cv_std = None
        #
        if cv_now:
            self.cross_validate()

    def cross_validate(self, X=None, y=None, kfolds=5):
        '''
        Perform cross-validation and return results.

        Args:
          X:
            Optional; Training data to perform CV on. Otherwise use X from object
          y:
            Optional; Training data to perform CV on. Otherwise use y from object
          kfolds:
            Optional; Number of folds for CV (default is 10)
        '''

        cv_X = X if X else self.X
        cv_y = y if y else self.y
        # R2 Results:
        self.cv_r2_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds)
        self.cv_r2_mean = np.mean(self.cv_r2_results)
        self.cv_r2_median = np.median(self.cv_r2_results)
        self.cv_r2_std = np.std(self.cv_r2_results)

        # MSE Results:
        self.cv_mse_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds, scoring ='mean_squared_error')
        self.cv_mse_mean = np.mean(self.cv_mse_results)
        self.cv_mse_median = np.median(self.cv_mse_results)
        self.cv_mse_std = np.std(self.cv_mse_results)

        #### Other Metrics
        #self.cv_neg_log_loss = cross_val_score(self.model, cv_X, cv_y, scoring ='neg_log_loss')
        #self.cv_log_loss = -(self.cv_neg_log_loss.mean())
        #self.cv_mse = cross_val_score(self.model, cv_X, cv_y, scoring ='mean_squared_error')
        #self.cv_mae = cross_val_score(self.model, cv_X, cv_y, scoring ='mean_absolute_error')
        #self.cv_rms = cross_val_score(self.model, cv_X, cv_y, scoring ='mean_squared_error', squared=False)
        #self.cv_r2 = cross_val_score(self.model, cv_X, cv_y, scoring ='r2')


        # To-Do: Claire Sarraille: Added precision, recall, and F-Measure:

    def print_cv_summary(self):
        cv_summary = (
        f'''CV Results for `{self.name}` model:
            R2: {self.cv_r2_mean:.5f} ± {self.cv_r2_std:.5f}
            Mean Squared Error: {self.cv_mse_mean} ± {self.cv_mse_std:.5f}
            10-fold Cross Validation Scores: {self.cv_r2_results}
        ''')
        print(cv_summary)


    def plot_cv(self, ax):
        '''
        Plot the cross-validation values using the array of results and given
        Axis for plotting.
        '''
        ax.set_title(f'CV Results for `{self.name}` Model')
        # Claire Sarraille: Customized plot for cv results
        sns.boxplot(self.cv_results, ax=ax)

        return ax

# Train Test Split

In [57]:
# Create X (predictors) and y (target) variables:
X = df.drop(['price'], axis = 1).reset_index(drop=True)
y = df.price.reset_index(drop=True)

# Split Data into train and test:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=666)

# Pipeline Set-up

### Boolean features don't need to be scaled
- so we will use 'remainder='passthrough'' To indicate to compiler to let them through pipeline without transformations.

In [70]:
boolean_cols = []
for col in range(X.shape[1]):
  nunique_vals = X.iloc[:,col].nunique()
  if nunique_vals == 2:
    boolean_cols.append(X.columns[col])

In [71]:
num_cols = []
for col in range(X.shape[1]):
  if X[X.columns[col]].dtypes != 'object' and X.columns[col] not in boolean_cols:
    num_cols.append(X.columns[col])

In [72]:
print(obj_cols)

['room_type', 'city', 'state']


In [73]:
print(num_cols)

['latitude', 'longitude', 'accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'bathrooms', 'desc_polarity', 'desc_subjectivity']


In [74]:
print(boolean_cols)

['host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'license_bool', 'is_shared_bathroom', 'air_conditioning', 'kitchen', 'wifi', 'free_park_prem', 'pets_allowed', 'pool']


## Pipeline for numerics

In [77]:
# Pipeline for numerics (SCALE data):

numpipe = Pipeline([
    ('ss', StandardScaler())
])


## Pipeline for Categoricals:

In [81]:
# Pipeline for categorical feature (One-Hot-Encode):

catpipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore')) # Using handle_unknown param en lieu of stratify during train-test-split
])

## We now have our numeric and categorical pipelines.
- Since we don't have any nulls, and binary features don't need to be transformed or one-hot-encoded,
- Our binary features (`international plan` and `voice mail plan`) don't need any transformations, so we will use `remainder='passthrough'` to let them through.
- Next step is to handle columns holistically with `ColumnTransformer`

In [91]:
# We will use our pipeline objects as transformer argument for ColumnTransformer
# our transformer is a tuple and includes a columns argument

ColTrans = ColumnTransformer(transformers=[
    ('numerics', numpipe, num_cols),
    ('categoricals', catpipe, obj_cols)
], remainder='passthrough')

# Model #1: Baseline Dummy

In [92]:
# Create Dummy Model Pipeline

dumpipe = Pipeline([
    ('ct', ColTrans),
    ('dummy', DummyRegressor(strategy='mean'))
])

In [93]:
# Fit the dummy regressor to the training data:
dumpipe.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categoricals',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['room_type', 'city',
                                                   'state'])])),
                ('dummy', DummyRegressor())])

In [94]:
print(f'Dummy model TRAIN Mean Accuracy score: {dumpipe.score(X_train, y_train):.5f}')

Dummy model TRAIN Mean Accuracy score: 0.00000
